In [ ]:
!pip install flask-ngrok

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Usar curl o Postman (app o extension chrome) para POST

*   Entrenar 
curl -X GET http://10a02b8a.ngrok.io/train

*   Borrar el modelo
curl -X GET http://10a02b8a.ngrok.io/wipe
*   Predecir

curl -i -X POST -H 'Content-Type: application/json' -d '  [ {"sepal.length" : 5.1, "sepal.width" : 3.5,"petal.length" : 1.4,"petal.width" : 0.2}, {"sepal.length" : 5.1, "sepal.width" : 3.5,"petal.length" : 1.4,"petal.width" : 0.2} ]' http://10a02b8a.ngrok.io/predict 

{"prediction":["Setosa","Setosa"]}



Descarga de curl:
windows:
https://curl.haxx.se/windows/

linux incluido (o se instala con administrador de paquetes)

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import sys
import os
import shutil
import time
import traceback

import pandas as pd
from sklearn.externals import joblib



app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

# inputs
training_data = '/content/gdrive/My Drive/Colab Notebooks/data/iris.csv'
features = ["sepal.length","sepal.width","petal.length","petal.width"]
target_class = ["variety"]

model_directory = '/content/gdrive/My Drive/Colab Notebooks/model'
model_file_name = '%s/model.pkl' % model_directory

clf= None


@app.route('/predict', methods=['POST'])
def predict():
    if clf:
        try:
            json_ = request.json

            query = pd.DataFrame(json_)
            print(query)
            
            prediction = list(clf.predict(query))

            print(prediction)
            return jsonify({"prediction": list(prediction)})

        except Exception as e:

            return jsonify({'error': str(e), 'trace': traceback.format_exc()})
    else:
        print('train first')
        return 'no model here'


@app.route('/train', methods=['GET'])
def train():
    # using random forest as an example
    # can do the training separately and just update the pickles
    from sklearn.ensemble import RandomForestClassifier as rf

    df = pd.read_csv(training_data)
    x = df[features]
    y = df[target_class]

    print(x)
    print(y)

    global clf
    clf = rf()
    start = time.time()
    clf.fit(x, y)

    joblib.dump(clf, model_file_name)

    message1 = 'Trained in %.5f seconds' % (time.time() - start)
    message2 = 'Model training score: %s' % clf.score(x, y)
    return_message = 'Success. \n{0}. \n{1}.'.format(message1, message2) 
    return return_message


@app.route('/wipe', methods=['GET'])
def wipe():
    try:
        shutil.rmtree(model_directory)
        os.makedirs(model_directory)
        return 'Model wiped'

    except Exception as e:
        print(str(e))
        return 'Could not remove and recreate the model directory'



@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"

try:
        clf = joblib.load(model_file_name)
        print('model loaded')
        
except Exception as e:
        print('No model here')
        print('Train first')
        print(str(e))
        clf = None

app.run()